#### 1.Introduction
This assignment will help you to consolidate the concepts learnt in the session.

### 2.Problem Statement 28

In this assignment, students will be using the K-nearest neighbors algorithm to predict
**how many points NBA players scored in the 2013-2014 season.**<br>
A look at the data before we dive into the algorithm, let’s take a look at our data. Each row in the data
contains information on how a player performed in the 2013-2014 NBA season.

Download 'nba_2013.csv' file from this link:
https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0

Here are some selected columns from the data:
- player - name of the player
- pos - the position of the player
- g - number of games the player was in
- gs - number of games the player started
- pts - total points the player scored

There are many more columns in the data, mostly containing information about average
player game performance over the course of the season. See this site for an explanation
of the rest of them.

We can read our dataset in and figure out which columns are present:


In [29]:
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas
with open("nba_2013.csv", 'r') as csvfile:
     nba = pandas.read_csv(csvfile)

In [3]:
nba.head(5)

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [6]:
nba.isnull().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [9]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
player          481 non-null object
pos             481 non-null object
age             481 non-null int64
bref_team_id    481 non-null object
g               481 non-null int64
gs              481 non-null int64
mp              481 non-null int64
fg              481 non-null int64
fga             481 non-null int64
fg.             479 non-null float64
x3p             481 non-null int64
x3pa            481 non-null int64
x3p.            414 non-null float64
x2p             481 non-null int64
x2pa            481 non-null int64
x2p.            478 non-null float64
efg.            479 non-null float64
ft              481 non-null int64
fta             481 non-null int64
ft.             461 non-null float64
orb             481 non-null int64
drb             481 non-null int64
trb             481 non-null int64
ast             481 non-null int64
stl             481 non-null int64
blk    

In [11]:
nba.describe()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
count,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,479.000000,481.000000,481.000000,414.000000,...,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.0
mean,26.509356,53.253638,25.571726,1237.386694,192.881497,424.463617,0.436436,39.613306,110.130977,0.285111,...,55.810811,162.817048,218.627859,112.536383,39.280665,24.103950,71.862786,105.869023,516.582121,2013.0
std,4.198265,25.322711,29.658465,897.258840,171.832793,368.850833,0.098672,50.855639,132.751732,0.157633,...,62.101191,145.348116,200.356507,131.019557,34.783590,30.875381,62.701690,71.213627,470.422228,0.0
min,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.0
25%,23.000000,32.000000,0.000000,388.000000,47.000000,110.000000,0.400500,0.000000,3.000000,0.234355,...,12.000000,43.000000,55.000000,20.000000,9.000000,4.000000,21.000000,44.000000,115.000000,2013.0
50%,26.000000,61.000000,10.000000,1141.000000,146.000000,332.000000,0.438000,16.000000,48.000000,0.330976,...,35.000000,135.000000,168.000000,65.000000,32.000000,14.000000,58.000000,104.000000,401.000000,2013.0
75%,29.000000,76.000000,54.000000,2016.000000,307.000000,672.000000,0.479500,68.000000,193.000000,0.375000,...,73.000000,230.000000,310.000000,152.000000,60.000000,32.000000,108.000000,158.000000,821.000000,2013.0
max,39.000000,83.000000,82.000000,3122.000000,849.000000,1688.000000,1.000000,261.000000,615.000000,1.000000,...,440.000000,783.000000,1114.000000,721.000000,191.000000,219.000000,295.000000,273.000000,2593.000000,2013.0


In [12]:
nba.describe(include=['O'])

,player,pos,bref_team_id,season
count,481,481,481,481
unique,481,7,31,1
top,Kyle Lowry,SG,TOT,2013-2014
freq,1,109,63,481


In [19]:
data = nba.drop(['pts','season','player'],axis=1)

In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
cat_cols = X.dtypes.loc[X.dtypes == 'object'].index

for cols in cat_cols:
    data = data.join(pd.get_dummies(data[cols],prefix=cols).iloc[:,:-1])

In [24]:
cat_cols

Index(['pos', 'bref_team_id'], dtype='object')

In [27]:
X = data.drop(['pos','bref_team_id'],axis=1)
y = nba.pts

In [28]:
imp = Imputer(missing_values="NaN", strategy='mean', axis=0)
X = imp.fit_transform(X)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

#### Building a Base model

In [46]:
for K in range(25):
    K_value = K+1
    neigh = KNeighborsRegressor(n_neighbors = K_value, weights='uniform', algorithm='auto')
    neigh.fit(X_train, y_train) 
    y_pred = neigh.predict(X_test)
    print("Root Mean Squared Error is ", np.sqrt(mean_squared_error(y_test,y_pred)),"for K-Value:",K_value)

Root Mean Squared Error is  93.50563324354239 for K-Value: 1
Root Mean Squared Error is  92.08431294012662 for K-Value: 2
Root Mean Squared Error is  87.23943798666912 for K-Value: 3
Root Mean Squared Error is  83.35868953318338 for K-Value: 4
Root Mean Squared Error is  79.84875012516929 for K-Value: 5
Root Mean Squared Error is  80.53674942324675 for K-Value: 6
Root Mean Squared Error is  80.21229572605384 for K-Value: 7
Root Mean Squared Error is  81.5920327362573 for K-Value: 8
Root Mean Squared Error is  82.56990691201301 for K-Value: 9
Root Mean Squared Error is  81.97995339484336 for K-Value: 10
Root Mean Squared Error is  85.18013701783266 for K-Value: 11
Root Mean Squared Error is  89.17085552519829 for K-Value: 12
Root Mean Squared Error is  92.51450483781802 for K-Value: 13
Root Mean Squared Error is  94.06893163347246 for K-Value: 14
Root Mean Squared Error is  94.19445770356644 for K-Value: 15
Root Mean Squared Error is  95.0079370833627 for K-Value: 16
Root Mean Squared E

Since error is minimum at **K=5** ...hence we have the best model at k=5

#### Improving the model performance...

In [38]:
from sklearn.preprocessing import RobustScaler, StandardScaler
rb = RobustScaler()
X_scaled = rb.fit_transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 100)

In [42]:
for K in range(25):
    K_value = K+1
    neigh = KNeighborsRegressor(n_neighbors = K_value, weights='uniform', algorithm='auto')
    neigh.fit(X_train, y_train) 
    y_pred = neigh.predict(X_test)
    print("Root Mean Squared Error is ", np.sqrt(mean_squared_error(y_test,y_pred)),"for K-Value:",K_value)

Root Mean Squared Error is  141.5149803207152 for K-Value: 1
Root Mean Squared Error is  123.40846810490761 for K-Value: 2
Root Mean Squared Error is  108.93017553351527 for K-Value: 3
Root Mean Squared Error is  105.43794138145253 for K-Value: 4
Root Mean Squared Error is  109.98875303004142 for K-Value: 5
Root Mean Squared Error is  110.37233657938177 for K-Value: 6
Root Mean Squared Error is  112.90821317509493 for K-Value: 7
Root Mean Squared Error is  114.57267494838194 for K-Value: 8
Root Mean Squared Error is  114.01601278202523 for K-Value: 9
Root Mean Squared Error is  111.40438361956485 for K-Value: 10
Root Mean Squared Error is  113.35920382090328 for K-Value: 11
Root Mean Squared Error is  115.41019775223819 for K-Value: 12
Root Mean Squared Error is  117.2820398964348 for K-Value: 13
Root Mean Squared Error is  118.98577996516968 for K-Value: 14
Root Mean Squared Error is  120.88274715354122 for K-Value: 15
Root Mean Squared Error is  122.45926332690787 for K-Value: 16
Roo

Since error is minimum at **K=4** ...hence we have the best model at k=4